In [21]:
#import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [22]:
tot_df = []
# for j,i in enumerate(np.arange(2016, 2021, 1)):
for j,i in enumerate(np.arange(2016, 2021, 1)):
    tot_df.append(pd.read_csv('../Data/Raw/CDC/CDCSTDcountysexrace'+ 
                              str(i) +'.csv', sep = ',', skiprows = 4))
df = pd.concat(tot_df, axis = 0)
# df = pd.read_csv('../Data/Raw/CDC/CDCSTDcountysexrace2016.csv', skiprows = 4)

In [23]:
df.head()

,Indicator,Year,Geography,FIPS,Race/Ethnicity,Sex,Cases,Rate per 100000
0,Primary and Secondary Syphilis,2016,"Abbeville County, SC",45001,White,Male,Data not available,Data not available
1,Primary and Secondary Syphilis,2016,"Abbeville County, SC",45001,White,Female,Data not available,Data not available
2,Primary and Secondary Syphilis,2016,"Abbeville County, SC",45001,Unknown,Male,Data not available,Data not available
3,Primary and Secondary Syphilis,2016,"Abbeville County, SC",45001,Unknown,Female,Data not available,Data not available
4,Primary and Secondary Syphilis,2016,"Abbeville County, SC",45001,Native Hawaiian/Other Pacific Islander,Male,Data not available,Data not available


In [24]:
df.columns= df.columns.str.lower()
df.columns = df.columns.str.replace(' ','_')
df.columns = df.columns.str.replace('/','_')
df.head()

,indicator,year,geography,fips,race_ethnicity,sex,cases,rate_per_100000
0,Primary and Secondary Syphilis,2016,"Abbeville County, SC",45001,White,Male,Data not available,Data not available
1,Primary and Secondary Syphilis,2016,"Abbeville County, SC",45001,White,Female,Data not available,Data not available
2,Primary and Secondary Syphilis,2016,"Abbeville County, SC",45001,Unknown,Male,Data not available,Data not available
3,Primary and Secondary Syphilis,2016,"Abbeville County, SC",45001,Unknown,Female,Data not available,Data not available
4,Primary and Secondary Syphilis,2016,"Abbeville County, SC",45001,Native Hawaiian/Other Pacific Islander,Male,Data not available,Data not available


In [25]:
# df = df.rename(columns = {'Cases':'cases',
#                             'Rate per 100000':'ratePer100000',
#                              'Indicator':'indicator',
#                              'Year':'year',
#                              'Geography':'geography',
#                              'FIPS':'fips',
#                              'Percent':'percent',
#                              'Sex':'sex',
#                              'Race/Ethnicity' : 'raceEthnicity'})
# df.head()

In [26]:
df.dtypes

indicator          object
year               object
geography          object
fips                int64
race_ethnicity     object
sex                object
cases              object
rate_per_100000    object
dtype: object

In [27]:
df.loc[df["year"].str.startswith('2020',na=False), "year"] = "2020"

In [28]:
data_to_be_removed = ["cases", "rate_per_100000"]

In [29]:
for col in data_to_be_removed:
    df.loc[df[col].str.contains("Data",na = False), col] = np.nan
    df[col] = df[col].str.replace(',','')

In [30]:
df.head()

,indicator,year,geography,fips,race_ethnicity,sex,cases,rate_per_100000
0,Primary and Secondary Syphilis,2016,"Abbeville County, SC",45001,White,Male,NaN,NaN
1,Primary and Secondary Syphilis,2016,"Abbeville County, SC",45001,White,Female,NaN,NaN
2,Primary and Secondary Syphilis,2016,"Abbeville County, SC",45001,Unknown,Male,NaN,NaN
3,Primary and Secondary Syphilis,2016,"Abbeville County, SC",45001,Unknown,Female,NaN,NaN
4,Primary and Secondary Syphilis,2016,"Abbeville County, SC",45001,Native Hawaiian/Other Pacific Islander,Male,NaN,NaN


In [31]:
df[["year","fips","cases","rate_per_100000"]] = df[["year","fips","cases","rate_per_100000"]].apply(pd.to_numeric)

In [32]:
df.drop(columns = 'geography', inplace = True)

In [33]:
df.dtypes

indicator           object
year                 int64
fips                 int64
race_ethnicity      object
sex                 object
cases              float64
rate_per_100000    float64
dtype: object

In [34]:
df = df.pivot_table(index=['year','fips'], columns=['sex', 'indicator','race_ethnicity']).reset_index()
df.columns = df.columns.map('_'.join).str.strip('_')
df.head()

,year,fips,cases_Female_Chlamydia_American Indian/Alaska Native,cases_Female_Chlamydia_Asian,cases_Female_Chlamydia_Black/African American,cases_Female_Chlamydia_Hispanic/Latino,cases_Female_Chlamydia_Multiracial,cases_Female_Chlamydia_Native Hawaiian/Other Pacific Islander,cases_Female_Chlamydia_Unknown,cases_Female_Chlamydia_White,...,rate_per_100000_Male_Gonorrhea_Multiracial,rate_per_100000_Male_Gonorrhea_Native Hawaiian/Other Pacific Islander,rate_per_100000_Male_Gonorrhea_White,rate_per_100000_Male_Primary and Secondary Syphilis_American Indian/Alaska Native,rate_per_100000_Male_Primary and Secondary Syphilis_Asian,rate_per_100000_Male_Primary and Secondary Syphilis_Black/African American,rate_per_100000_Male_Primary and Secondary Syphilis_Hispanic/Latino,rate_per_100000_Male_Primary and Secondary Syphilis_Multiracial,rate_per_100000_Male_Primary and Secondary Syphilis_Native Hawaiian/Other Pacific Islander,rate_per_100000_Male_Primary and Secondary Syphilis_White
0,2019,1001,0.0,1.0,43.0,2.0,0.0,0.0,143.0,34.0,...,0.0,0.0,39.7,0.0,0.0,38.7,0.0,0.0,0.0,0.0
1,2019,1003,0.0,0.0,19.0,10.0,1.0,0.0,439.0,68.0,...,0.0,0.0,12.2,0.0,0.0,10.7,18.0,0.0,0.0,2.2
2,2019,1005,0.0,0.0,99.0,3.0,8.0,0.0,31.0,16.0,...,653.6,0.0,101.8,0.0,0.0,79.9,0.0,0.0,0.0,0.0
3,2019,1007,0.0,0.0,14.0,2.0,0.0,0.0,59.0,14.0,...,0.0,0.0,11.8,0.0,0.0,34.3,0.0,0.0,0.0,0.0
4,2019,1009,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,0.0,0.0,69.4,0.0,0.0,0.0,0.0,0.0,0.0,4.1


In [35]:
df.columns= df.columns.str.lower()
df.columns = df.columns.str.replace(' ','_')
df.columns = df.columns.str.replace('/','_')
df.head()

,year,fips,cases_female_chlamydia_american_indian_alaska_native,cases_female_chlamydia_asian,cases_female_chlamydia_black_african_american,cases_female_chlamydia_hispanic_latino,cases_female_chlamydia_multiracial,cases_female_chlamydia_native_hawaiian_other_pacific_islander,cases_female_chlamydia_unknown,cases_female_chlamydia_white,...,rate_per_100000_male_gonorrhea_multiracial,rate_per_100000_male_gonorrhea_native_hawaiian_other_pacific_islander,rate_per_100000_male_gonorrhea_white,rate_per_100000_male_primary_and_secondary_syphilis_american_indian_alaska_native,rate_per_100000_male_primary_and_secondary_syphilis_asian,rate_per_100000_male_primary_and_secondary_syphilis_black_african_american,rate_per_100000_male_primary_and_secondary_syphilis_hispanic_latino,rate_per_100000_male_primary_and_secondary_syphilis_multiracial,rate_per_100000_male_primary_and_secondary_syphilis_native_hawaiian_other_pacific_islander,rate_per_100000_male_primary_and_secondary_syphilis_white
0,2019,1001,0.0,1.0,43.0,2.0,0.0,0.0,143.0,34.0,...,0.0,0.0,39.7,0.0,0.0,38.7,0.0,0.0,0.0,0.0
1,2019,1003,0.0,0.0,19.0,10.0,1.0,0.0,439.0,68.0,...,0.0,0.0,12.2,0.0,0.0,10.7,18.0,0.0,0.0,2.2
2,2019,1005,0.0,0.0,99.0,3.0,8.0,0.0,31.0,16.0,...,653.6,0.0,101.8,0.0,0.0,79.9,0.0,0.0,0.0,0.0
3,2019,1007,0.0,0.0,14.0,2.0,0.0,0.0,59.0,14.0,...,0.0,0.0,11.8,0.0,0.0,34.3,0.0,0.0,0.0,0.0
4,2019,1009,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,0.0,0.0,69.4,0.0,0.0,0.0,0.0,0.0,0.0,4.1


In [36]:
pd.unique(df['year'])

array([2019, 2020])

In [37]:

df.info(memory_usage = "deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6185 entries, 0 to 6184
Columns: 122 entries, year to rate_per_100000_male_primary_and_secondary_syphilis_white
dtypes: float64(120), int64(2)
memory usage: 5.8 MB


In [38]:
df['fips'].value_counts()

1001     2
39071    2
39123    2
39125    2
39127    2
        ..
21179    2
21181    2
20071    1
42093    1
4011     1
Name: fips, Length: 3094, dtype: int64

In [39]:
df[df['fips']==30037]

,year,fips,cases_female_chlamydia_american_indian_alaska_native,cases_female_chlamydia_asian,cases_female_chlamydia_black_african_american,cases_female_chlamydia_hispanic_latino,cases_female_chlamydia_multiracial,cases_female_chlamydia_native_hawaiian_other_pacific_islander,cases_female_chlamydia_unknown,cases_female_chlamydia_white,...,rate_per_100000_male_gonorrhea_multiracial,rate_per_100000_male_gonorrhea_native_hawaiian_other_pacific_islander,rate_per_100000_male_gonorrhea_white,rate_per_100000_male_primary_and_secondary_syphilis_american_indian_alaska_native,rate_per_100000_male_primary_and_secondary_syphilis_asian,rate_per_100000_male_primary_and_secondary_syphilis_black_african_american,rate_per_100000_male_primary_and_secondary_syphilis_hispanic_latino,rate_per_100000_male_primary_and_secondary_syphilis_multiracial,rate_per_100000_male_primary_and_secondary_syphilis_native_hawaiian_other_pacific_islander,rate_per_100000_male_primary_and_secondary_syphilis_white
1570,2019,30037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4662,2020,30037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,...,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
df.to_csv("../Data/Clean/CDC/disagRaceSexSTD2019_2020_clean.csv",index=False)